# CBC News (Web Scraping)

Library Import

In [1]:
import sqlite3
import os
import pandas as pd
from bs4 import BeautifulSoup
import requests
from pandasql import sqldf
import time
from datetime import datetime as dt

Helper Function for Database

In [2]:
def execSQL(conn,query):
  conn.execute(query) # execute an SQL query
  conn.commit() # "commit" that query in order to make its action permanent

pysqldf = lambda q: sqldf(q, globals())  # Querying Pandas Dataframes using SQL

Create Database

In [3]:
# Remove the database if it is already here
try:
  os.remove("cbcnews.db")
  print("remove earlier copy of cbcnews.db")
except:
  print("not already here")  

# create new one
conn = sqlite3.connect("cbcnews.db")

# enable foreign keys
execSQL(conn,"PRAGMA foreign_keys=ON")

remove earlier copy of cbcnews.db


Create Tables in Database

- Table 1
    - store the data id
    - conventient for checking the duplicate data id
- Table 2
    - set foreign key, linkage with the Table 1
    - update data fields 
        1. data id
        2. title
        3. description
        4. link
        5. timestamp

In [4]:
# Table 1: id_table
createCMD='''
CREATE TABLE id_table( 
  DataID TEXT PRIMARY KEY
)
'''

execSQL(conn,createCMD)

In [5]:
# Table 2: content_table
createCMD='''
CREATE TABLE content_table(
  DataID TEXT PRIMARY KEY,
  Title  TEXT,
  Description  TEXT,
  Link  TEXT,
  TimeStamp  TEXT,
  FOREIGN KEY(DataID) REFERENCES id_table(DataID)
)
'''

execSQL(conn,createCMD)

Web Scraping

In [6]:
def grad_data_id():
    # grab the database
    df_content = pd.read_sql_query("SELECT * FROM id_table", conn)
    exist_id_list = df_content['DataID'].to_list()
    return exist_id_list

In [7]:
def scraping(link):
    resp = requests.get(link)
    # Exceptional case
    if resp.status_code != 200:
        print(f'Invalid Status Code: {resp.status_code}')
        return False
    soup = BeautifulSoup(resp.text, 'lxml')
    # Get primary and secondary stories
    primary_stories = soup.find('div', {'class': 'primaryTopStories'})
    secondary_stories = soup.find('div', {'class': 'secondaryTopStories'})
    
    # Split the stories into each cell of ResultSet (just like list)
    primary_story_items = primary_stories.findAll('a', {'data-test': 'type-story'})
    secondary_story_items = secondary_stories.findAll('a', {'data-test': 'type-story'})
    
    # Update exist id list
    exist_id_list = grad_data_id()
    
    # Grab data fields of each story
    # Alternate method: write a function to replace two for-loops
    
    # Primary Stories
    for item_num in range(len(primary_story_items)):
        item_id = primary_story_items[item_num].attrs['data-contentid']
        item_link = primary_story_items[item_num].attrs['href']
        item_title = primary_story_items[item_num].find('h3', {'class': 'headline'}).text
        item_description = primary_story_items[item_num].find('div', {'class': 'description'}).text
        item_timestamp = primary_story_items[item_num].find('time', {'class': 'timeStamp'}).attrs['datetime']
        # check data id with the first table(id_table) in database
        if not (item_id in exist_id_list):
            # after checking, there is a new story
            new_item = pd.DataFrame({'DataID': item_id, 'Title': item_title, 
                                    'Description': item_description, 'Link': item_link, 
                                    'TimeStamp': item_timestamp}, index=[0])
            new_id = pd.DataFrame({'DataID': item_id}, index=[0])
            # update to database
            new_id.to_sql('id_table', conn, if_exists='append', index=False)
            new_item.to_sql('content_table', conn, if_exists='append', index=False)
            print(f'{dt.strptime(item_timestamp, "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d %H:%M:%S")}\n{item_title}\n')
            # tg_bot_message(f'{dt.strptime(item_timestamp, "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d %H:%M:%S")}\n{item_title}')

    # Secondary Stories
    for item_num in range(len(secondary_story_items)):
        item_id = secondary_story_items[item_num].attrs['data-contentid']
        item_link = secondary_story_items[item_num].attrs['href']
        item_title = secondary_story_items[item_num].find('h3', {'class': 'headline'}).text
        item_timestamp = secondary_story_items[item_num].find('time', {'class': 'timeStamp'}).attrs['datetime']
        item_description = None
        # check data id with the first table(id_table) in database
        if not (item_id in exist_id_list):
            # after checking, there is a new story
            new_item = pd.DataFrame({'DataID': item_id, 'Title': item_title, 
                                    'Description': item_description, 'Link': item_link, 
                                    'TimeStamp': item_timestamp}, index=[0])
            new_id = pd.DataFrame({'DataID': item_id}, index=[0])
            # update to database
            new_id.to_sql('id_table', conn, if_exists='append', index=False)
            new_item.to_sql('content_table', conn, if_exists='append', index=False)
            print(f'{dt.strptime(item_timestamp, "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d %H:%M:%S")}\n{item_title}\n')
            # tg_bot_message(f'{dt.strptime(item_timestamp, "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d %H:%M:%S")}\n{item_title}')
    return True

In [8]:
def tg_bot_message(message):
    chat_id = None
    token = ''
    payload = {'chat_id': chat_id, 'text': message}
    requests.get(f'https://api.telegram.org/bot{token}/sendMessage', data=payload)

Main

In [9]:
# Target Link
url = 'https://www.cbc.ca/news/canada/calgary'
flag = True
while flag:
    flag = scraping(url)
    if not (flag):
        print('Stop Scraping')
        break
    time.sleep(300) # check news every 5 mins

2023-02-27 12:00:00
Alberta's big budget question: What to do with the torrent of cash flooding provincial coffers

2023-02-27 17:41:22
Alberta, Ottawa sign health-care funding deal worth $24B over 10 years

2023-02-27 15:20:08
Canmore residents long for more permanent neighbours as 'weekenders' snap up homes

2023-02-27 13:31:15
Battleground Calgary: Alberta NDP move election campaign HQ to city's core

2023-02-26 17:52:27
From well cleanups to Sovereignty Act, Danielle Smith's big ideas keep deflating

2023-02-27 13:00:00
Edmonton has seen more opioid-related EMS calls than Calgary for past 2 years

2023-02-27 12:00:00
Want to know how COVID-19 subvariants are behaving in Alberta? Now it's a bit easier

2023-02-27 09:00:00
Lock the doors. Get straight home. I live in fear because of hate crimes committed against others

2023-02-27 20:25:53
Statistics Canada study on Black-owned businesses suggests systemic challenges hold them back

2023-02-26 15:00:00
How this facility in central Al

KeyboardInterrupt: 